<a href="https://colab.research.google.com/github/Agiseri/Image-Classifier/blob/main/Image_Classifier2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multiclass Classification | Caltech101 |Tensorflow
Hi everyone! In this my new notebook we're going to learn how to predict classes of objects with given images coords in dataset, using VGG16 (CNN) by Tensorflow. 

### 🥰 Acknowledgements
[Multi-class object detection and bounding box regression with Keras, TensorFlow, and Deep Learning](https://pyimagesearch.com/2020/10/12/multi-class-object-detection-and-bounding-box-regression-with-keras-tensorflow-and-deep-learning/) by Adrian Rosebrock.

# 📚 Libraries
Firslty, we need to import custom library *imutils* by Adrian Rosebrock. It will be imported from folders that were loaded by be to Kaggle:

In [ ]:
# adding imutils to our virtual environment
import sys
sys.path.append('../input/imutils-054/imutils-0.5.4')

Also, we'll use *os, opencv, matplotlib, numpy, tensorflow, sklearn, etc*.

In [ ]:
import imutils
import os
import cv2
import datetime
import numpy as np
import random

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.models import load_model
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dropout
from tensorflow.keras.utils import to_categorical
from  tensorflow.keras.callbacks import ModelCheckpoint

from sklearn.preprocessing import LabelBinarizer

import pickle

# 🖍 Initialize variables
These variables will store out input data, target labels and also names of image files.

In [ ]:
data = []
labels = []
imagePaths = []

# 📖 Load dataset (images, classes)
Annotation file has classes (names of folders). 

In [ ]:
images_path = "../input/caltech101-airplanes-motorbikes-schooners/caltech101_classification"

In [ ]:
classes = ["Motorbikes", "airplanes", "schooner"]

In [ ]:
# counts number of images in each class
def classes_counter(labels, class_name):
    counter = 0
    for l in labels:
        if l == class_name:
            counter += 1
    return counter

In [ ]:
for cl in classes:
    images_list = [] 
    
    path_new = images_path + "/" + cl + "/"
    print(path_new)
    
    # get the list of the available images
    for image in os.listdir(path_new): 
        # get only images that 
        # are located in folder
        if (image.endswith(".jpg")):
            images_list.append(image)
    
    # sort image_path in ascending order
    images_list = sorted(images_list)  
        
    # loop over the images
    for img in images_list:
        label = cl
        
        image_path = os.path.sep.join([images_path, cl, img])
        image = cv2.imread(image_path)
        (h, w) = image.shape[:2]
        
        # load the image
        image = load_img(image_path, target_size=(224, 224))
        image = img_to_array(image)
        
        data.append(image)
        labels.append(label)
        imagePaths.append(image_path)

../input/caltech101-airplanes-motorbikes-schooners/caltech101_classification/Motorbikes/


FileNotFoundError: ignored

In [ ]:
# show the output image
imgplot = plt.imshow(image.astype('uint8'))
plt.show()

# 🔧 Data preparation

Let's check **how many images** are in each class.

In [ ]:
counter_mtb = classes_counter(labels, "Motorbikes")
counter_arp = classes_counter(labels, "airplanes")
counter_sch = classes_counter(labels, "schooner")

counter_mtb, counter_arp, counter_sch

Here we get the **maximum value** of number of images.

In [ ]:
max_number = max(counter_mtb, counter_arp, counter_sch)

As we can see, we don't have so much shooners, so we **need to augment** them. Also, I think, we'll create two more pictures of motorbikes in order to have also 800 pics. We'll do **scaling and rotating**.

In [ ]:
def make_scale(img):
    # scale range
    scale_val = random.uniform(0.8, 1.2)
    imgScaled = cv2.resize(img.copy(), 
                           None, 
                           fx=scale_val, 
                           fy=scale_val)
    
    return imgScaled

In [ ]:
def make_rotate(img):
    (h, w) = img.shape[:2]
    
    # degrees range
    rotate_val = random.uniform(-5, 5)
    
    # image center
    center = (w / 2, h / 2)  
    
    # Rotation Matrix
    M = cv2.getRotationMatrix2D(center, 
                                rotate_val, 
                                scale=1)
    
    imgRotated = cv2.warpAffine(img.copy(), 
                                M, 
                                (w, h))
    return imgRotated

Also, we need to check the number of images in each class in order to equalize number of images in each class. That's why we'll do an [**augmentation**](https://neptune.ai/blog/data-augmentation-in-python).

In [ ]:
def augment_data(counter, max_number, class_name):
    
    # while we don't have a lot of images
    while counter < max_number:
        # loop through each image in list
        
        for img in data:
            # check the number of images again
            
            if counter < max_number:
                # make scaling
                imgAug = img.copy()
                imgAug = make_scale(imgAug)
                
                # temporary save the new image
                cv2.imwrite("imgAug.jpg", imgAug)
                
                # load the new image
                imgAug = load_img("imgAug.jpg", target_size=(224, 224))
                imgAug = img_to_array(imgAug)
                
                # delete it from memory
                os.remove("imgAug.jpg")
                
                # add new image, it's label and path
                data.append(imgAug)
                labels.append(class_name)
                imagePaths.append(image_path)
                
                # recalculate a counter
                counter = classes_counter(labels, class_name)
            else:
                break

            # make rotating
            if counter < max_number:
                imgAug = img.copy()
                imgAug = make_rotate(imgAug)
                
                # temporary save the new image
                cv2.imwrite("imgAug.jpg", imgAug)
                
                # load the new image
                imgAug = load_img("imgAug.jpg", target_size=(224, 224))
                imgAug = img_to_array(imgAug)
                
                # delete it from memory
                os.remove("imgAug.jpg")
                
                # add new image and it's label and path
                data.append(imgAug)
                labels.append(class_name)
                imagePaths.append(image_path)
                
                # recalculate a counter
                counter = classes_counter(labels, class_name)
            else:
                break

Let's apply the augmentation to **"Motorbikes" and "schooner"** classes.

In [ ]:
augment_data(counter_mtb, max_number, "Motorbikes")
augment_data(counter_sch, max_number, "schooner")

Let's check **how many images** are in each class after augmentation.

In [ ]:
counter_mtb = classes_counter(labels, "Motorbikes")
counter_arp = classes_counter(labels, "airplanes")
counter_sch = classes_counter(labels, "schooner")

counter_mtb, counter_arp, counter_sch

As you can see, now we have an **equal number of images** in each class.

Also, we need to **normalize data** (convert from range [0, 255] to [0, 1]).

In [ ]:
# convert from the range [0, 255] to [0, 1]
data = np.array(data, dtype="float32") / 255.0

Let's convert everything else to **numpy arrays** also.

In [ ]:
# convert to numpy array
labels = np.array(labels)
imagePaths = np.array(imagePaths)

After that we **convert our class labels** ("butterfly", "dalmatian", "dolphin") to **[one-hot encoding](https://en.wikipedia.org/wiki/One-hot)**.

In [ ]:
# one-hot encoding on the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

And, in general, we **need to check**: if it is binary classification (two classes) or multiclass classification (three or more classes).

In [ ]:
if len(lb.classes_) == 2:
    print("two classes")
    labels = to_categorical(labels)

Here we **divide data to train and test sets**. I decided to divide into 95% to 5% respectively.

In [ ]:
split = train_test_split(data,
                         labels,
                         imagePaths,
                         test_size=0.05,
                         random_state=42)

And **unpack** *split* variable to different variables.

In [ ]:
# unpack the data split
(trainImages, testImages) = split[:2]
(trainLabels, testLabels) = split[2:4]
(trainPaths, testPaths) = split[4:]

Also, we can **save names of test images** in a * *.txt file* in order to test neural network later.

In [ ]:
f = open("testing_multiclass.txt", "w")
f.write("\n".join(testPaths))
f.close()

# 🧠 Neural Network Architecture
Here we'll use VGG16 neural network.

In [ ]:
vgg = VGG16(weights="imagenet",
            include_top=False,
            input_tensor=Input(shape=(224, 224, 3)))

In [ ]:
# freeze all layers of VGG 
# in order not to train them
vgg.trainable = False

In [ ]:
# flatten the max-pooling 
# output of VGG
flatten = vgg.output
flatten = Flatten()(flatten)

And for **class prediction (classification task)** we'll use a softmax [**activation function**](https://en.wikipedia.org/wiki/Softmax_function).

In [ ]:
# construct a second fully-connected 
# layer header to predict
# the class label

softmaxHead = Dense(512, activation="relu")(flatten)
softmaxHead = Dropout(0.5)(softmaxHead)
softmaxHead = Dense(512, activation="relu")(softmaxHead)
softmaxHead = Dropout(0.5)(softmaxHead)

softmaxHead = Dense(len(lb.classes_), 
                    activation="softmax", 
                    name="class_label")(softmaxHead)

Finally, we need to **add this output** to our VGG16 model.

In [ ]:
model = Model(
    inputs=vgg.input,
    outputs=(softmaxHead))

Also, we need to define some **hyperparameters** *(learning rate, number of epochs, size of batch)*.

In [ ]:
INIT_LR = 1e-4
NUM_EPOCHS = 40
BATCH_SIZE = 32

Then we define a dictionary to set the **loss method**: *categorical crossentropy* for the class label.

In [ ]:
losses = {
    "class_label": "categorical_crossentropy",
}

We need to construct a dictionary for our **target training output**.

In [ ]:
trainTargets = {
    "class_label": trainLabels,
}

We need to construct a second dictionary, this one for our **target testing output**.

In [ ]:
testTargets = {
    "class_label": testLabels,
}

Also, we would like to **save only the best model** from all epochs:

In [ ]:
model_path = "model.h5"

model_checkpoint_callback = ModelCheckpoint(
    filepath=model_path,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In the end, we **initialize the optimizer, compile the model, and show the model summary**.

In [ ]:
opt = Adam(INIT_LR)

model.compile(loss=losses, 
              optimizer=opt, 
              metrics=["accuracy"])

print(model.summary())

# 🐎 Train Neural Network & Save best model
Here we **train our VGG16 network** for class label prediction.

In [ ]:
H = model.fit(
    trainImages, trainTargets,
    validation_data=(testImages, testTargets),
    batch_size=BATCH_SIZE,
    epochs=NUM_EPOCHS,
    callbacks=[model_checkpoint_callback],
    verbose=1)

# 💾 Save label binarizer

In [ ]:
f = open("lb.pickle", "wb")
f.write(pickle.dumps(lb))
f.close()

# 📊 Visualize the results
Here we'll visualize **loss and accuracy**.

In [ ]:
lossNames = ["loss"]

N = np.arange(0, NUM_EPOCHS)
plt.style.use("ggplot")
plt.figure(figsize=(17, 10))


# plot the loss for both the training and validation data
plt.title("Loss & Val Loss")
plt.xlabel("Epoch №")
plt.ylabel("Loss")
plt.plot(N, H.history["loss"], label="loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
plt.legend()

plt.show()

In [ ]:
# create a new figure for the accuracies
plt.style.use("ggplot")
plt.figure(figsize=(17, 10))

plt.plot(N, H.history["accuracy"], label="acc")
plt.plot(N, H.history["val_accuracy"], label="val_acc")

plt.title("Accuracy & Val Accuracy")
plt.xlabel("Epoch №")
plt.ylabel("Accuracy")
plt.legend()

# 📋 Test the model
Let's **load filenames of test images**.

In [ ]:
path = "testing_multiclass.txt"
filenames = open(path).read().strip().split("\n")
imagePaths = []

for f in filenames:
    imagePaths.append(f)

Let's **load the VGG16 model and label binarizer**.

In [ ]:
model = load_model("./model.h5")

In [ ]:
lb = pickle.loads(open("./lb.pickle", "rb").read())

Let's **predict class of test images**.

In [ ]:
# counter for viewing images
cntr = 0

for imagePath in imagePaths:

    # load the input image
    image = load_img(imagePath, target_size=(224, 224))
    image = img_to_array(image) / 255.0
    image = np.expand_dims(image, axis=0)

    # predict classes
    (labelPreds) = model.predict(image)

    # determine the class label 
    # with the largest predicted
    # probability
    i = np.argmax(labelPreds, axis=1)
    label = lb.classes_[i][0]

    # load the input image (in OpenCV format)
    image = cv2.imread(imagePath)
    image = imutils.resize(image, width=600)
    (h, w) = image.shape[:2]



    # show the output image
    print("class label = ", label)
    imgplot = plt.imshow(cv2.cvtColor(image, 
                                      cv2.COLOR_BGR2RGB).astype('uint8'))
    plt.show()
    
    # increment counter
    cntr += 1
    
    # view only first 10 
    # test images
    if (cntr > 10):
        break

# 🧷 Conclusion
As you can see, our model makes predictions quite correctly! That is great! That is all and thak you for your attetion.

### 💖 If you liked it - please, make an upvote!
### 💌 Subscribe to my YouTube Channel: [LearnAI](https://www.youtube.com/channel/UCEJ8IRbmEl3tEZahc17pwrw)
### 📗 My notebooks on the similar topic: 
* #### [Stop Signs Detection | Bounding Box Regression](https://www.kaggle.com/code/maricinnamon/stop-signs-detection-bounding-box-regression)
* #### [Bounding Box Regression & Object Classification](https://www.kaggle.com/code/maricinnamon/bounding-box-regression-object-classification)